# Demand Analysis I

## Preliminaries

In [2]:
# Load libraries
import numpy as np
import pandas as pd
import HWDataProcessing as dp
import EstimationTools as et
from EstimationTools import Model

In [3]:
# Load data
dataFile = "data/Shining32.csv"
data = pd.read_csv(dataFile)

# Process data
data = dp.CleanDataSet(data)

# Take a look
data.head()

,buyerid,age,education,inc,gender,location,purchase,brandid,mint,white,...,loc_Queens,gen_Female,gen_Male,gen_Other,purchase_InStore,purchase_Online,brand_Aquafresh,brand_Colgate,brand_Crest,brand_Sensodyne
0,1,56,College,0.079523,Female,Brooklyn,Online,Colgate,1,1,...,0,1,0,0,0,1,0,1,0,0
1,1,56,College,0.079523,Female,Brooklyn,Online,Colgate,1,0,...,0,1,0,0,0,1,0,1,0,0
2,1,56,College,0.079523,Female,Brooklyn,Online,Colgate,0,1,...,0,1,0,0,0,1,0,1,0,0
3,1,56,College,0.079523,Female,Brooklyn,Online,Crest,1,1,...,0,1,0,0,0,1,0,0,1,0
4,1,56,College,0.079523,Female,Brooklyn,Online,Crest,0,0,...,0,1,0,0,0,1,0,0,1,0


In [62]:
a = np.array(range(64)).reshape((16,4),order='F')
b = a.reshape((8,2,4),order='F')
c= b.mean(axis=1)

e = b.reshape((2,2,4,4),order='F')
a,e.mean(axis=0).reshape((8,4),order='F')
a,c


(array([[ 0, 16, 32, 48],
        [ 1, 17, 33, 49],
        [ 2, 18, 34, 50],
        [ 3, 19, 35, 51],
        [ 4, 20, 36, 52],
        [ 5, 21, 37, 53],
        [ 6, 22, 38, 54],
        [ 7, 23, 39, 55],
        [ 8, 24, 40, 56],
        [ 9, 25, 41, 57],
        [10, 26, 42, 58],
        [11, 27, 43, 59],
        [12, 28, 44, 60],
        [13, 29, 45, 61],
        [14, 30, 46, 62],
        [15, 31, 47, 63]]),
 array([[ 4., 20., 36., 52.],
        [ 5., 21., 37., 53.],
        [ 6., 22., 38., 54.],
        [ 7., 23., 39., 55.],
        [ 8., 24., 40., 56.],
        [ 9., 25., 41., 57.],
        [10., 26., 42., 58.],
        [11., 27., 43., 59.]]))

In [31]:
a = np.array(range(16)).reshape((8,2),order='F')
a

array([[ 0,  8],
       [ 1,  9],
       [ 2, 10],
       [ 3, 11],
       [ 4, 12],
       [ 5, 13],
       [ 6, 14],
       [ 7, 15]])

In [9]:
a=data[["Chosen","product"]].groupby(by="product").mean()
a.shape

(10, 1)

In [14]:
a = data[(~ data["SecondChoice"].isnull()) &((data["Chosen"]==1) | (data["SecondChoice"]==1))]
a[["buyerid","product","Chosen","SecondChoice"]].head(30)

,buyerid,product,Chosen,SecondChoice
10,2,6,0,1.0
19,2,1,1,0.0
125,13,10,0,1.0
127,13,3,1,0.0
170,18,6,1,0.0
174,18,7,0,1.0
610,62,6,1,0.0
615,62,10,0,1.0
641,65,5,1,0.0
647,65,3,0,1.0


In [50]:
def unpackParameters(theta,k,ro,ru,nuPosition):
    betaBar = theta[:k,:].reshape((k,1))
    betaO = theta[k:(k+k*ro),:].reshape((k,ro),order='F')
    betaUp = theta[(k+k*ro):,:]
    betaU = np.zeros((k,1))
    betaU[nuPosition,:] = betaUp
    
    return betaBar, betaO, betaU

k = 4
ro =2
nuPosition = np.array([1,0,0,1]) == 1
ru = sum(nuPosition)
theta = 3*np.ones((k+k*ro+ru,1))

betaBar, betaO, betaU = unpackParameters(theta,k,ro,ru,nuPosition)
betaBar, betaO, betaU

(array([[3.],
        [3.],
        [3.],
        [3.]]),
 array([[3., 3.],
        [3., 3.],
        [3., 3.],
        [3., 3.]]),
 array([[3.],
        [0.],
        [0.],
        [3.]]))

In [48]:
nuPosition = np.array([1,0,0,1]) == 1
nuPosition

array([ True, False, False,  True])

## Simple logit model estimation

In [3]:
# Model specification
J = 10     # number of products
yName = "Chosen"
xName = ["mint","white","fluoride","kids"]  # product characteristics
zName = []              # consumer attributes
pName = "priceperpack"  # product price
brands = "brandid"      # brand identifier
iv = ["discount"]

logitMod = Model(type='logit',data=data,J=J,y=yName,x=xName,z=zName,
                p=pName,brands = brands,iv=iv)

In [4]:
estimates,vcv = logitMod.fitLogit()
estimates,vcv,np.sqrt(np.diag(vcv))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 34
         Function evaluations: 222
         Gradient evaluations: 37
         Current function value: 0.000000
         Iterations: 7
         Function evaluations: 144
         Gradient evaluations: 22


(array([ 1.89389967, -1.0634944 ,  0.28448555, -1.02661954, -2.31469652]),
 array([[ 0.15988221,  0.05801044,  0.03512713,  0.16004204, -0.17914266],
        [ 0.05801044,  0.02248345,  0.01362962,  0.05817277, -0.06816386],
        [ 0.03512713,  0.01362962,  0.00876988,  0.03496936, -0.04010268],
        [ 0.16004204,  0.05817277,  0.03496936,  0.16666963, -0.18128019],
        [-0.17914266, -0.06816386, -0.04010268, -0.18128019,  0.2281882 ]]),
 array([0.39985273, 0.14994481, 0.09364762, 0.40825192, 0.47769049]))

In [5]:
elasticities= logitMod.getElasticityLogit(estimates)
elasticities

array([[ -8.76876708,   2.90049744,   0.02690765,   0.89604464,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,  -6.7007548 ,   0.02690765,   0.89604464,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744, -10.92671049,   0.89604464,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744,   0.02690765,  -8.81924594,
          0.09924563,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744,   0.02690765,   0.89604464,
        -10.53720839,   0.93813528,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  0.91796478,   2.90049744,   0.02690765,   0.89604464,
          0.09924563, -10.00598566,   0.04876691,   2.91828575,
          0.02463005,   0.15089887],
       [  

In [ ]:
a = np.array([[1,2,3],[1,2,3],[1,2,3],[1,2,3]])
b = a.reshape((2,6),order='F')
b

In [ ]:
a = np.array([1,2,3,4,5,6])[:,np.newaxis].reshape((3,2),order='F')
b = np.kron(np.ones((7,7)),a)-np.kron(a,np.ones((7,7)))
b

In [ ]:
np.kron(np.ones((7,1)),a)


In [ ]:
np.kron(a,np.ones((7,1)))

In [ ]:
c = np.array([[1,5],[2,6],[3,7],[4,8]])
c.reshape((2,4),order='F').reshape((4,2),order='F')

In [ ]:
c

In [ ]:
c.reshape(-1,2,c.shape[-1]).sum(1)